In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


## let's import some astropy modules
from astropy.io import fits #  to handle FITS files
from astropy.coordinates import SkyCoord # conversion of RA/Dec and other astronomical coordinates
import astropy.units as u # allows you to attach units/dimensions to numbers in a calculation
import astropy.constants as c # stores values and dimensions of major constants
from astropy.wcs import WCS, utils # handling of the world coordinate system information in a FITS header

Astropy has many useful functions for astronomy/astrophysics. Because astropy is open source, there are **many** contributors all over the world, some of whom have dedicated themselves to replacing beloved software (like _iraf_)  that's nearing obsolescense.

We'll start by exploring two of astropy's convenience functions -- _astropy.units_ and _astropy.constants_. These modules allow you to easily convert between units and make sure your results are dimensionally correct. 

# Unit conversions + constants in calculations

Let's solve a quick problem using _astropy.constants_ and _astropy.units_. Say we want to derive the mass of the Galaxy from its circular velocity in the solar neighborhood.

At a radius of 8.5 kpc, the circular velocity of the Milky Way is $v_c \sim 220$ km/s. 

Using the equation $v_c = \sqrt{\frac{GM}{r}}$, solve for the mass of the Galaxy in solar units.

We'll start by defining our given values and attaching units to them:

In [ ]:
r = 8.5*u.kpc
v_c = 220*u.km/u.s

M = r * v_c**2/ c.G

M

You can see that "M" now carries units, but they're kind of strange and some of them should cancel. We can use the function _.decompose()_ to reduce the units.

In [ ]:
M.decompose()

Okay, that's a little better! There are no extraneous units, **but** we want mass in solar masses. Now we can use _.to()_ to convert between kg and $M_\odot$.

In [ ]:
M.to(u.Msun)

Now let's do a more complicated one -- plot the blackbody curve for a source with a temperature of 6000 K with the correct units. Feel free to do this in either frequency or wavelength space.

For many applications including (especially!) dealing with radio astronomy data, using _.constants_ and _.units_ will be big time savers! (That said, it's still useful to understand dimensional analysis and how to convert between units.)

***

# Basic FITS file handling with astropy

While DS9 and MaxIm DL are terrific for viewing, and interacting with, astronomical data through a GUI, often analysis of those data and figure generation are best handled within your own specialized code. Within python, there are multiple ways to read in FITS files, _astropy.io.fits_ is one (lightweight) standard option.

Go ahead and download the two FITS files you'll need for this tutorial [here](https://drive.google.com/drive/folders/1cwc1d-VTUGFbDdQ_ZJ69-GjZl3I4utx0?usp=sharing).

We'll start by actually reading in our data (details of this field [here](https://ui.adsabs.harvard.edu/abs/2021ApJ...914L..23P/abstract)). Assuming that everything is downloaded to the same directory, you can leave the path information as it is. If that's not the case, though, you will need to edit the string that directs the _fits.open()_ to your FITS file.

In [ ]:
hdu = fits.open('jboa61uzq_flc.fits')

hdu

We'll call our generically open FITS file "hdulist" in reference to what is printed with *hdulist.info()*, the list of Header Data Units. (Ultimately, the verbiage matters less than understanding what you've actually read in, so -- as  always -- name your variables in a way that helps **you**.) The different HDUs will have different embedded headers and data. To call one, you can say _hdulist\[{index}\]_ -- let's see what the header information is for the "PRIMARY" HDU. To view the header, simply use the _.header_ command.

Can you identify the telescope, instrument, and filter with which the data was taken? (***Answer: HST, ACS, F814W***)

In [ ]:
hdulist[0].header

Okay, now that we know something about these data, we should actually view an image. Let's look at the "SCI" HDU. Within matplotlib, we can view array-like data with *plt.imshow*, FITS **data** can be accessed via the _.data_ command. We'll need to define a colormap and dynamic range. The _np.percentile()_ calls in _vmin_ and _vmax_ set that range based on the data itself.

In [ ]:
fig = plt.figure(figsize = (15, 10))
plt.imshow(hdulist[1].data, cmap = 'gray_r', vmin = np.percentile(hdulist[1].data, 5), 
           vmax = np.percentile(hdulist[1].data, 95), origin = 'lower')
plt.axis('off')

Now you try -- there's another FITS file in this directory. These data are from the Legacy Survey and cutouts are downloaded not as multi-extension FITS files, but as data cubes. You can use normal array manipulation to access the slice you want. Open it (make sure to check the header!) and show the r-band image. You should carefully select your colormap and 

***

# WCS from FITS headers

The World Coordinate System (WCS) information that's stored in the header of FITS files is used to relate the native coordinates of the frame (pixels) to coordinates on the sky. While calculations can be done by hand (well, by computer) to relate those different coordinate systems, _astropy.wcs_ can handle most operations for you via built-in functions. Let's start by plotting adding a grid and coordinate labels to our DECaLS image. The function _astropy.wcs.WCS_ will allow us to access all of the WCS information in the image header and then project the data accordingly.

Because the Legacy Survey data comes in a data cube, we will have to use the additional command _.slice()_ to ensure we're accessing the correct header information. The argument "view" will be the cube slice you actually want to view.

In [ ]:
legacy_wcs = WCS(dat[0].header, dat)

fig = plt.figure(figsize = (10, 10))
plt.subplot(projection=legacy_wcs.slice(view = [1]))
plt.imshow(dat[0].data[1, :, :], cmap = 'gray_r', vmin = np.percentile(dat[0].data[1, :, :], 0.5), 
           vmax = np.percentile(dat[0].data[1, :, :], 99.5), origin = 'lower')
plt.grid(color='gray', ls='dashed')
plt.xlabel('Right Ascension', fontsize = 15)
plt.ylabel('Declination', fontsize = 15)

Why don't you try to plot the HST data with a grid over it below?

We can also access information about the coordinates at a particular pixel location using _astropy.wcs.utils.pixel_to_skycoord_ (and vice versa with *.skycoord_to_pixel*). Let's identify the coordinates at the center of the HST image. We'll use the shape of the image to determine the location of the central pixel.

In [ ]:
cen_coords = utils.pixel_to_skycoord(hdulist[1].data.shape[0]/2, hdulist[1].data.shape[1]/2, 
                                     wcs = HST_wcs)

cen_coords

The result is a SkyCoord object ([documentation here](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)), which can be manipulated to display the coordinates in different systems (like galactic coordinates) or forms (i.e., sexagesimal or degrees).

***

# Exercise

Based on the pixel scale of the initial HST image we opened (0.05 arcsec/pix) calculate the dimensions of the image in degrees. For the same image, use the header information to find the pixel location for the RA/Dec 10:00:30.03 +02:08:59.47 then show the image and mark the location of these coordinates.